<a href="https://colab.research.google.com/github/wSanice/etl_vendas_varejo_olist_sheets.ipynb/blob/main/etl_vendas_varejo_olist_sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d olistbr/brazilian-ecommerce
!unzip brazilian-ecommerce.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
  0% 0.00/42.6M [00:00<?, ?B/s]
100% 42.6M/42.6M [00:00<00:00, 1.08GB/s]
Archive:  brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


In [ ]:
import pandas as pd

df_orders = pd.read_csv('olist_orders_dataset.csv')
df_items = pd.read_csv('olist_order_items_dataset.csv')
df_products = pd.read_csv('olist_products_dataset.csv')
df_customers = pd.read_csv('olist_customers_dataset.csv')
df_merged = pd.merge(df_orders, df_customers, on='customer_id')
df_merged = pd.merge(df_merged, df_items, on='order_id')
df_merged = pd.merge(df_merged, df_products, on='product_id')
df_merged['order_purchase_timestamp'] = pd.to_datetime(df_merged['order_purchase_timestamp'])
df_2018 = df_merged[df_merged['order_purchase_timestamp'].dt.year == 2018].copy()

colunas_finais = [
    'order_id',
    'order_purchase_timestamp',
    'price',
    'freight_value',
    'product_category_name',
    'customer_state',
    'customer_city'
]

df_final = df_2018[colunas_finais]

print(f"Tamanho do DataFrame final: {df_final.shape}")

Tamanho do DataFrame final: (61416, 7)


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

sh = gc.create('Analise_Varejo_Olist_2018')

df_final_sem_nan = df_final.fillna('')

if 'order_purchase_timestamp' in df_final_sem_nan.columns:
    df_final_sem_nan['order_purchase_timestamp'] = df_final_sem_nan['order_purchase_timestamp'].astype(str)

worksheet = sh.get_worksheet(0)
worksheet.update([df_final_sem_nan.columns.values.tolist()] + df_final_sem_nan.values.tolist())

print(f"Link para acesso: {sh.url}")

Link para acesso: https://docs.google.com/spreadsheets/d/1tUZLQqLxO3obK7nMH3b_4pIqfOW2FMw9F63Tvb3Vrec
